# (bstudent) notebook for musing, notes, demos

### (bstudent) Normal Equation and Gradient Descent:

$\hat\theta = \displaystyle\min_{\theta} \frac{1}{N} \mathbf{\epsilon}^T\mathbf{\epsilon}$  
  
$\hat\theta = \displaystyle\min_{\theta} \frac{1}{N} (\mathbf{\hat y} - \mathbf{y})^T (\mathbf{\hat y} - \mathbf{y})$  
  
$\mathbf{\hat y} = \mathbf{X}\mathbf{\theta}$

$\implies \nabla_\theta \frac{1}{N} (\mathbf{X}\mathbf{\theta}-\mathbf{y})^T(\mathbf{X}\mathbf{\theta}-\mathbf{y}) = \mathbf{0}$  

Minimum occurs where:  

$\nabla_\theta \frac{1}{N} (\mathbf{X}\mathbf{\theta}-\mathbf{y})^T(\mathbf{X}\mathbf{\theta}-\mathbf{y}) = \frac{1}{N} \left( 2 \mathbf{X}^T\mathbf{X}\mathbf{\theta} - 2 \mathbf{X}^T\mathbf{y}\right) = \mathbf{0}$  

$\implies \mathbf{X}^T\mathbf{X}\hat\theta = \mathbf{X}^T\mathbf{y}$  

$\implies \hat\theta = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$  
(The "normal equation")  

For general gradient at points other than $\hat\theta$, the "gradient" in MSE gradient descent is:    

$\nabla_\theta \frac{1}{N} \left(\mathbf{X}\mathbf{\theta}-\mathbf{y})^T(\mathbf{X}\mathbf{\theta}-\mathbf{y}\right) = \frac{2}{N} \mathbf{X}^T \left(\mathbf{X}\mathbf{\theta} - \mathbf{y} \right)$

In [2]:
import numpy as np
import pandas as pd

### demo np.c_

Shows that c has the nice property of turning vectors into columns, i.e. my intuitive bias was that the 0'th list element might need to have more than one dimension for this to work, but that is not the case!  

In [3]:
ones1 = np.ones((5,))
ones2 = 2*np.ones((5,))
ones3 = 3*np.ones((5,))

In [4]:
np.c_[ones1, ones2, ones3]

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

#### also demo np.r_

This result is counterintuitive, given the way np.c_ works above; IT is probably consistent in some way that would not elude a person smarter or more insightful than myself.

In [5]:
np.r_[ones1, ones2, ones3]

array([1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.])

In [7]:
ones1T = ones1.reshape((1,5))
ones1T

array([[1., 1., 1., 1., 1.]])

In [8]:
# np.r_[ones1T, ones2, ones3]

## Q: what happens when you call `list(...)` with a `dataframe` argument?

In [9]:
df = pd.DataFrame({'a': [1,2,3,4,5], 'b': list('uvwxy'), 'c': [5,10,15,20,25]})
df

,a,b,c
0,1,u,5
1,2,v,10
2,3,w,15
3,4,x,20
4,5,y,25


#### A: you get a list of the column names

In [10]:
list(df)

['a', 'b', 'c']

dict(...) on a data frame returns a dict of series:

In [11]:
dict(df)

{'a': 0    1
 1    2
 2    3
 3    4
 4    5
 Name: a, dtype: int64, 'b': 0    u
 1    v
 2    w
 3    x
 4    y
 Name: b, dtype: object, 'c': 0     5
 1    10
 2    15
 3    20
 4    25
 Name: c, dtype: int64}

In [13]:
type((dict(df)['a']))

pandas.core.series.Series

In [10]:
tups = list(zip(list(range(10)), np.array([[ 2.81585438,  7.09167958,  3.82972099,  0.79365551,  5.8885703, 
                        9.29718395,  1.79862509,  8.10392157, -0.228207  ,  4.83753243]]).flatten()))
tups

[(0, 2.81585438),
 (1, 7.09167958),
 (2, 3.82972099),
 (3, 0.79365551),
 (4, 5.8885703),
 (5, 9.29718395),
 (6, 1.79862509),
 (7, 8.10392157),
 (8, -0.228207),
 (9, 4.83753243)]


BTW, can VSCode jupyter notebook render math?  Let's find out:  

$x^{(j)} = \sum_{i=1}^N y_i^{(j)}$